In [1]:
# Dependencies

import pandas as pd
import numpy as np
from citipy import citipy
from config import weather_api_key
import requests
import time
from datetime import datetime

In [2]:
# Generating random sets of latitude and longitudes.

lats = np.random.uniform(-90,90, size=2000)
lngs = np.random.uniform(-180,180, size=2000)

coordinates = list(zip(lats,lngs))

In [3]:
# Create a list for holding the cities.

cities = []

# Using citipy in coordinates.

for coordinate in coordinates:
    city = (citipy.nearest_city(coordinate[0],coordinate[1]).city_name)
    
    if city not in cities:
        cities.append(city)
        
len(cities)

772

In [4]:
# Creating URL API to make the hits.

url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [5]:
# Create an empty list to hold the weather data.

city_data = []

# Print the beginning of the logging.

print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.

record_count = 1
set_count = 1

# Loop through all the cities in the list.

for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count.
    
    record_count += 1
# Run an API request for each of the cities.

    try:
        # Parse the JSON and retrieve data.
        
        city_weather = requests.get(city_url).json()
        
        # Parse out the needed data.
        
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
        
        # Convert the date to ISO standard.
        
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Append the city information into city_data list.
        
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date, 
                          "Description": city_description})

# If an error is experienced, skip the city.

    except:
        
        print("City not found. Skipping...")
        
        pass

# Indicate that Data Loading is complete.

print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | cape town
Processing Record 2 of Set 1 | port alfred
Processing Record 3 of Set 1 | barrow
Processing Record 4 of Set 1 | rikitea
Processing Record 5 of Set 1 | ilulissat
Processing Record 6 of Set 1 | chumikan
Processing Record 7 of Set 1 | attawapiskat
City not found. Skipping...
Processing Record 8 of Set 1 | mahebourg
Processing Record 9 of Set 1 | ushuaia
Processing Record 10 of Set 1 | vaini
Processing Record 11 of Set 1 | karaton
Processing Record 12 of Set 1 | kaka
Processing Record 13 of Set 1 | tiksi
Processing Record 14 of Set 1 | atuona
Processing Record 15 of Set 1 | faanui
Processing Record 16 of Set 1 | dikson
Processing Record 17 of Set 1 | provideniya
Processing Record 18 of Set 1 | klaksvik
Processing Record 19 of Set 1 | karakol
Processing Record 20 of Set 1 | busselton
Processing Record 21 of Set 1 | molchanovo
Processing Record 22 of Set 1 | hasaki
Processing Record 23 of Set

In [6]:
# Convert the array of dictionaries to a Pandas DataFrame.

city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
0,Cape Town,-33.9258,18.4232,66.11,37,0,5.75,ZA,2022-08-15 13:54:47,clear sky
1,Port Alfred,-33.5906,26.8910,61.18,63,87,4.00,ZA,2022-08-15 13:59:47,overcast clouds
2,Barrow,71.2906,-156.7887,32.52,98,100,16.28,US,2022-08-15 13:59:47,snow
3,Rikitea,-23.1203,-134.9692,73.22,74,1,21.16,PF,2022-08-15 13:59:48,clear sky
4,Ilulissat,69.2167,-51.1000,43.57,80,100,6.58,GL,2022-08-15 13:59:49,overcast clouds
5,Chumikan,54.7183,135.3150,53.85,97,97,4.38,RU,2022-08-15 13:59:49,light rain
6,Mahebourg,-20.4081,57.7000,73.69,60,40,19.57,MU,2022-08-15 13:59:50,scattered clouds
7,Ushuaia,-54.8000,-68.3000,35.26,87,40,6.91,AR,2022-08-15 13:57:43,scattered clouds
8,Vaini,-21.2000,-175.2000,71.76,78,40,10.36,TO,2022-08-15 13:59:51,scattered clouds
9,Karaton,-6.3424,106.1045,74.80,93,29,0.74,ID,2022-08-15 13:59:52,scattered clouds


In [7]:
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Description"]

city_data_df = city_data_df[new_column_order]

In [8]:
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,Cape Town,ZA,-33.9258,18.4232,66.11,37,0,5.75,clear sky
1,Port Alfred,ZA,-33.5906,26.8910,61.18,63,87,4.00,overcast clouds
2,Barrow,US,71.2906,-156.7887,32.52,98,100,16.28,snow
3,Rikitea,PF,-23.1203,-134.9692,73.22,74,1,21.16,clear sky
4,Ilulissat,GL,69.2167,-51.1000,43.57,80,100,6.58,overcast clouds


In [9]:
# Create the output file (CSV).

output_data_file = "WeatherPy_Database.csv"

# Export the City_Data into a CSV.

city_data_df.to_csv(output_data_file)